거스름돈의 금액을 나타내는 숫자가 주어졌을 때, 거스름돈을 낼 수 있는 동전의 최소 개수가 몇 개인지 구하는 함수를 작성하라. 

예) 미국 동전이 있다고 가정해보자: 1, 5, 10, 25 센트

makeChange(1) = 1 (1)

makeChange(2) = 2 (5+1)

makehange(49) = 7 (25 + 10 + 10 + 1 + 1 + 1 + 1)

## 첫 번째 해법 찾기

첫 번째 해법부터 찾아보자. 최대/최소/가장 큰 경우의 수/가장 작은 경우의 수를 찾는 문제를 풀 때 좋은 기술 중 하나는 가능한 모든 조합들을 비교해 보는 것이다. 비록 비효율적인 방법일 지 모르나 이 단계에서는 효율성을 고려하지 않아도 된다. 이 기술은 동적 해법을 찾는데 큰 도움이 된다.

우선, 가능한 모든 조합을 찾는 재귀함수를 간단히 작성해보자 (fig.7). 각 재귀 단계마다 해법은 하위 문제들로 나뉘어 질 수 있다. 만약 25센트가 선택된다면 남은 거스름돈을 지불하려면 최소 몇 개의 동전이 필요할까?

In [6]:
#Fig 7. 단순한 잔돈 계산 해법

# java
'''
//정말 단순한 방법. 합이 c가 되는 모든 동전의 조합을
//구하여, 가장 최소가 되는 숫자를 구한다.
private int[] coins  = new int[]{10, 5, 1};
public int makeChange(int c) {
   if (c == 0) return 0;
   int minCoins = Integer.MAX_VALUE;
   
   //총 액에서 각 동전의 값을 뺀 후
   //몇 개의 코인이 더 필요한 지 계산한다.
   for(int coin:coins) {
       
       //남은 액수보다 동전이 더 클 경우 넘어간다.
       if (c - coin >= 0) {
           int currMinCoins = makeChange(c - coin);
           if (currMinCoins < minCoins)
               minCoins = currMinCoins;
       }
   }
   
   //코인이 하나 줄어들 때마다 재귀적으로 더해지는 숫자
   return minCoins + 1;
}
'''
#python
import sys

coins = [10, 6, 1]
def makeChange(c):
    minCoins = sys.maxsize
    if (c is 0): return 0
    
    else:
        for coin in coins:
            if (c - coin) >= 0:
                currMinCoins = makeChange(c - coin)
                if(currMinCoins < minCoins):
                    minCoins = currMinCoins
                    
        return minCoins + 1
    
makeChange(15)

이 문제를 풀기 위해 자주 사용되는 효율적 알고리즘을 탐욕 알고리즘(greedy algorithm)이라 한다. 이 상황에서는 남은 거스름돈보다 크지 않은 가장 큰 동전을 찾아야 하기 때문이다. 하지만 이 방식은 다양한 동전 조합들을 구별해 낼 수 없다. (1, 6, 10 센트를 가지고 있을 때 $makeChange(12)$를 계산하는 경우를 생각해 보라.) 그러므로 더 일반화된 해법을 찾아내야 한다.

## 첫 번째 해법 분석

예상했듯이 코드를 실행해 보면 첫 번째 반복 부분부터 비효율적이라는 것을 알 수 있다. 가능한 모든 조합을 살펴보아야 하기 때문에 굉장히 많은 경우를 검토해야 한다.

![title](../img/dp_3.png "makeChange(12)의 재귀 트리(coins={10, 6, 1})")

<center>fig.8.makeChange(12)의 재귀 트리(coins={10, 6, 1})</center>


$makeChange(12)$의 실행에서 볼 수 있듯이 (fig. 8) 수형도는 최대 $c$의 높이를 가지고 각 단계에서 $n$개로 분기한다. 여기서 $n$이란 다른 숫자들의 경우의 수를 뜻한다. 즉 시간 복잡도를 big O 표기법으로 나타내면 $O(C^n)$이 된다. 수행시간이 너무 길고 재귀적인 구조인 이 해법을 동적 프로그래밍으로 해결해 볼 수 있는 지 확인해보자. 

<b>1. 최적 부분 구조</b>

위에서 보았듯이 이 문제는 재귀를 이용해서 풀기 적합하다. 또한 계산 순서가 바뀌더라도 결과가 변하지 않고(commutative) 결합 순서에도 영향을 받지 않으므로 (associative) 이 문제는 더 큰 하나의 문제로 묶일 수 있는 부분문제들, 즉 가장 적은 개수의 동전을 구하는 문제들로 이루어져 있다.

<b>2. 중복 부분 구조</b>

이 특성을 찾는 가장 좋은 방법은 수형도를 그린 후 여러 개의 분기들 중 겹치는 것이 있는 지 확인하는 것이다. 비록 위의 그림에서는 보이지 않지만 우리는 $makeChange(11)$이 $makeChange(1)$, $makeChange(5)$, $makeChange(10)$으로 나누어진다는 것을 알고 있다. $makeChange(1)$과 $makeChange(5)$는 다른 분기에서 호출되므로 이 둘은 중복된다는 것을 알 수 있다. 이러한 특성들을 고려하여 FAST 방식으로 이 문제의 동적 해법을 찾아보도록 할 것이다.

## 부분 문제 확인
각각의 함수는 각 코인을 한 번씩 재귀적으로 호출한다. 본래의 인풋은 작은 요소들로 쪼개어 지며 각각의 값을 갖고 있기 때문에 이 재귀적 호출들은 부분 문제들에 해당한다. 이 부분 문제들의 의미는 원래의 문제인 $makeChange(c)$와 동일하기 때문에 이해하기 쉽다. 즉, $makeChange(c)$라는 함수는 c가 어떤 값을 취하든 단순히 c 센트를 만드는 데 필요한 동전의 최소 개수를 반환한다. 그러므로 $makeChange(c-coin)$은 단순히 $c-coin$ 센트를 만들 수 있는 동전의 최소 개수를 의미한다는 것을 쉽게 알 수 있다.

이러한 이해를 바탕으로,기존 해법을 하향식의 동적 해법으로 쉽게 바꿀 수 있다. 결과들이 계산되는대로 캐시에 저장해준다. 즉, 다양한, 더 적은 금액의 거스름 돈을 지불하는 최소 동전의 개수를 캐싱해 나가는 것이다. 

## 해법 수정

## 결론